# Single-View Geometry (Python)

## Usage
This code snippet provides an overall code structure and some interactive plot interfaces for the *Single-View Geometry* section of Assignment 3. In [main function](#Main-function), we outline the required functionalities step by step. Some of the functions which involves interactive plots are already provided, but [the rest](#Your-implementation) are left for you to implement.

## Package installation
- You will need [GUI backend](https://matplotlib.org/faq/usage_faq.html#what-is-a-backend) to enable interactive plots in `matplotlib`.
- In this code, we use `tkinter` package. Installation instruction can be found [here](https://anaconda.org/anaconda/tk).

# Common imports

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os

from PIL import Image
import pickle

In [4]:
%matplotlib tk

# Provided functions

In [4]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.axis('off')
    plt.imshow(im)
    print(f'Set at least {min_lines} lines to compute vanishing point')
    print(f'The delete and backspace keys act like right clicking')
    print(f'The enter key acts like middle clicking')
    while True:
        print('Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print(f'Need at least {min_lines} lines, you have {n} now')
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers

In [40]:
def plot_lines_and_vp(ax, im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10
    
    ax.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        ax.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    ax.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    ax.set_xlim([bx1, bx2])
    ax.set_ylim([by2, by1])

# Your implementation

In [36]:
def get_vanishing_point(lines, num):
    """
    Solves for the vanishing point using the user-input lines.
    """
    # <YOUR CODE>
    H = np.zeros((num, 3))
    for i in range(num):
        H[i] = lines[:, i] 
    U, S, V = np.linalg.svd(H)
    
    return V[-1] / V[-1, -1]

In [35]:
def get_horizon_line(vanish_ptx, vanish_ptz):
    """
    Calculates the ground horizon line.
    """
    line = np.cross(vanish_ptx, vanish_ptz)
    scale = np.sqrt(line[0] ** 2 + line[1] ** 2)
    line  /= scale
    return line

In [34]:
def plot_horizon_line(ax, im, vp, horizontal_line):
    """
    Plots the horizon line.
    """
    # <YOUR CODE>
    bx1 = min(1, vp[0, 0] / vp[2, 0], vp[0, 2] / vp[2, 2]) - 10
    bx2 = max(im.shape[1], vp[0, 0] / vp[2, 0], vp[0, 2] / vp[2, 2]) + 10
    by1 = min(1, vp[1, 0] / vp[2, 0], vp[1, 2] / vp[2, 2]) - 10
    by2 = max(im.shape[0], vp[1, 0] / vp[2, 0], vp[1, 2] / vp[2, 2]) + 10
    ax.imshow(im)
    ax.plot(vp[0, 0] / vp[2, 0], vp[1, 0] / vp[2, 0], 'ro', label='Vanish point 1')
    ax.plot(vp[0, 2] / vp[2, 2], vp[1, 2] / vp[2, 2], 'ro', label='Vanish point 2')
    x_vals = np.array(ax.get_xlim())
    y_vals = (-horizontal_line[2] - horizontal_line[0] * x_vals) / horizontal_line[1]
    ax.plot(x_vals, y_vals, 'b')
    ax.set_xlim([bx1, bx2])
    ax.set_ylim([by2, by1])


In [33]:
def get_camera_parameters(vpts):
    """
    Computes the camera parameters. Hint: The SymPy package is suitable for this.
    """
    # <YOUR CODE>
    def get_param_mat(vpti, vptj):
        vptix, vptiy = vpti[0] / vpti[2], vpti[1] / vpti[2]
        vptjx, vptjy = vptj[0] / vptj[2], vptj[1] / vptj[2]
        A = np.array([[vptix * vptjx + vptiy * vptjy, vptix + vptjx, vptiy + vptjy, 1]])
        return A
    A = np.zeros((3, 4))
    A[0] = get_param_mat(vpts[:, 0], vpts[:, 1])
    A[1] = get_param_mat(vpts[:, 0], vpts[:, 2])
    A[2] = get_param_mat(vpts[:, 1], vpts[:, 2])        
    
    U, S, V = np.linalg.svd(A)
    intrinsic_ele = V[-1] 
    print(intrinsic_ele)
    # f = 1 / np.sqrt(intrinsic_ele[0])
    Cx = - intrinsic_ele[1] / intrinsic_ele[0] 
    Cy = - intrinsic_ele[2] / intrinsic_ele[0] 
    f = np.sqrt(- (Cx ** 2 + Cy ** 2) + 1 / intrinsic_ele[0])
    K = np.array([[f, 0, Cx], [0, f, Cy], [0, 0, 1]])
    
    return f, Cx, Cy, K
    

In [32]:
def get_rotation_matrix(K, vpts):
    """
    Computes the rotation matrix using the camera parameters.
    """
    # <YOUR CODE>
    R = np.zeros((3, 3))
    for i in range(3):
        R[:, i] =  (vpts[:, i] @ np.linalg.inv(K)).T
        R[:, i] /= np.linalg.norm(R[:, i])
    return R 

# Main function

In [5]:
im = np.asarray(Image.open('./city.jpg'))

# Also loads the vanishing line data if it exists in data.pickle file. 
# data.pickle is written using snippet in the next cell.
if os.path.exists('./data.pickle'):
    with open('./data.pickle', 'rb') as f:
        all_n, all_lines, all_centers = pickle.load(f)
    num_vpts = 3

In [11]:
# Click and save the line data for vanishing points. This snippet 
# opens up an interface for selecting points and writes them to 
# data.pickle file. The file is over-written.

num_vpts = 3
all_n, all_lines, all_centers = [], [], []

# for i in range(num_vpts):
#     print(f'Getting vanishing point {i}')
    
#     # Get at least three lines from user input
#     n_i, lines_i, centers_i = get_input_lines(im)
#     all_n.append(n_i)
#     all_lines.append(lines_i)
#     all_centers.append(centers_i)

# with open('data.pickle', 'wb') as f:
#     pickle.dump([all_n, all_lines, all_centers], f)

Getting vanishing point 0
Set at least 3 lines to compute vanishing point
The delete and backspace keys act like right clicking
The enter key acts like middle clicking
Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input
Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input
Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input
Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input
Click the two endpoints, use the right button (delete and backspace keys) to undo, and use the middle button to stop input
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
The delete and backspace keys act like right clicking
The enter key acts like middle clicking
Click the two endpoints, use the right button (de

In [12]:
for k in range(3):
    num = all_n[k]
    print("axis:", k)
    for i in range(num):
        print(f'Line {i}: {all_lines[k].shape} {all_lines[k][:, i]}; center: {all_centers[k][:, i]}')

axis: 0
Line 0: (3, 4) [   -29.11290323     88.87096774 -17638.46514048]; center: [232.40322581 274.60483871   1.        ]
Line 1: (3, 4) [  -114.91935484    289.59677419 -68175.27705515]; center: [334.2983871  368.07258065   1.        ]
Line 2: (3, 4) [   -16.85483871     91.93548387 -16450.05072841]; center: [229.33870968 220.97580645   1.        ]
Line 3: (3, 4) [  -142.5           281.93548387 -73305.94432882]; center: [335.06451613 429.36290323   1.        ]
axis: 1
Line 0: (3, 4) [-1.71612903e+02  6.12903226e+00  4.58495109e+04]; center: [278.37096774 313.67741935   1.        ]
Line 1: (3, 4) [-1.57822581e+02  7.66129032e+00  3.76045044e+04]; center: [253.08870968 305.25         1.        ]
Line 2: (3, 4) [-1.51693548e+02  1.22580645e+01  3.09276899e+04]; center: [227.80645161 296.05645161   1.        ]
Line 3: (3, 4) [-1.44032258e+02  6.12903226e+00  2.79306920e+04]; center: [206.35483871 292.22580645   1.        ]
axis: 2
Line 0: (3, 4) [ 8.58064516e+01  1.17983871e+02 -9.16277

In [13]:
# Part (1)
# Computing vanishing points for each of the directions
vpts = np.zeros((3, num_vpts))

for i in range(num_vpts):
    fig = plt.figure(); ax = fig.gca()
    
    # <YOUR CODE> Solve for vanishing point
    vpts[:, i] = get_vanishing_point(all_lines[i], all_n[i])
    
    # Plot the lines and the vanishing point
    plot_lines_and_vp(ax, im, all_lines[i], vpts[:, i])
    fig.savefig('Q3_vp{:d}.png'.format(i), bbox_inches='tight')

In [14]:
# Part (2) Computing and plotting the horizon
# <YOUR CODE> Get the ground horizon line
horizon_line = get_horizon_line(vpts[:, 0], vpts[:, 2])

# <YOUR CODE> Plot the ground horizon line
fig = plt.figure(); ax = fig.gca()
plot_horizon_line(ax, im, vpts, horizon_line)
fig.savefig('Q3_horizon.png', bbox_inches='tight')

In [69]:
# Part (3) Computing Camera Parameters
# <YOUR CODE> Solve for the camera parameters (f, u, v)
f, u, v, K = get_camera_parameters(vpts)
print(u, v, f)
print(K)

[ 1.87336099e-06 -8.08721643e-04 -4.69475120e-04  9.99999563e-01]
431.69557101058683 250.60579448786265 533.5125280980743
[[533.5125281    0.         431.69557101]
 [  0.         533.5125281  250.60579449]
 [  0.           0.           1.        ]]


In [53]:
# Part (4) Computing Rotation Matrices
# <YOUR CODE> Solve for the rotation matrix
R = get_rotation_matrix(K, vpts)
print(R)

[[-3.20701562e-03  4.25035914e-04  2.10473943e-03]
 [ 1.56971287e-03  3.26142442e-03  3.71824181e-04]
 [ 9.99993626e-01 -9.99994591e-01 -9.99997716e-01]]


## Extra Credits

In [46]:
def detect_lines(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    BlurGrayImage = cv2.GaussianBlur(gray, (5, 5), 1)
    # Generating Edge image
    EdgeImage = cv2.Canny(BlurGrayImage, 40, 255)
    # Create Line Segment Detector
    lsd = cv2.createLineSegmentDetector(0)

    # Detect line segments
    dlines = lsd.detect(EdgeImage)[0]
    detected_lines = [np.zeros((3, 0))] * 3

    # Draw lines on the image
    if dlines is not None:
        for dline in dlines:
            x1, y1, x2, y2 = map(int, dline.flatten())
            angle = np.arctan2(y2 - y1, x2 - x1) * 180.0 / np.pi
            length = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
            if length > 40:
                pt1 = np.array([x1, y1, 1])
                pt2 = np.array([x2, y2, 1])
                line = np.cross(pt1, pt2).reshape((3, 1))
                if angle > 0 and angle < 30:
                    if detected_lines[0].shape[1] < 5:
                        detected_lines[0] = np.append(detected_lines[0], line, axis=1)
                elif (angle < 95 and angle > 85) or (angle > -95 and angle < -85):
                    if detected_lines[1].shape[1] < 5:
                        detected_lines[1] = np.append(detected_lines[1], line, axis=1)
                elif  (angle < -20 and angle > -50):
                    if detected_lines[2].shape[1] < 5:
                        detected_lines[2] = np.append(detected_lines[2], line, axis=1)
    return detected_lines

In [47]:
img = cv2.imread('./city.jpg')
detected_lines = detect_lines(img)

In [48]:
vpts_new = np.zeros((3, num_vpts))

for i in range(num_vpts):
    fig = plt.figure(); ax = fig.gca()
    
    # <YOUR CODE> Solve for vanishing point
    vpts_new[:, i] = get_vanishing_point(detected_lines[i], detected_lines[i].shape[1])
    
    # Plot the lines and the vanishing point
    plot_lines_and_vp(ax, img, detected_lines[i], vpts_new[:, i])
    fig.savefig('Q3_extra_vp{:d}.png'.format(i), bbox_inches='tight')